## Find external Bias values to control HDSoCv2 self trigger behavior

The HDSoCv2 has an issue where the trigger threshold and reference voltages are dependent on the number of channels used. \
16 channels is the maximum number of channels with useful slope on the trigger values. 

A method to work around the issue is to set the trigger threshold and reference values to a fixed value. Then use the external bias to set the baseline. \
The normal way is to set the baseline then move the trigger threshold, however doing the opposite works too.

The key to controlling the baseline in relation to the trigger threshold is to know the voltage leve for a fixed trigger value.

This notebook allows a user to get that value.

### Imports

In [ ]:
%load_ext autoreload
%autoreload 2
import time
import datetime
import pickle
import pathlib

import polars as pl
from matplotlib import pyplot as plt

from naludaq.board import Board
from naludaq.communication import AnalogRegisters

### Directory Setup

In [ ]:
DATA_DIR = pathlib.Path().cwd() / "data"
DATA_DIR.mkdir(exist_ok=True, parents=True)
READOUT_DIR = pathlib.Path().cwd() / "readouts"
READOUT_DIR.mkdir(exist_ok=True, parents=True)

### Function Definitions

In [ ]:
def set_references(board: Board, ref_range: tuple[int, int]):
    board.trigger.references = {
        "0_15": ref_range,
        "16_31": ref_range,
        "32_47": ref_range,
        "48_63": ref_range,
    }


def set_ch_en(board: Board, channels: dict[int, bool]):
    """Enable/Disable specific channels."""
    CH_EN_BIT = 2
    board.trigger._update_fwd_regs(channels, CH_EN_BIT)


def set_tsel(board: Board, channels: dict[int, bool]):
    """Enable/Disable tsel for specific channels."""
    TSEL_BIT = 3
    board.trigger._update_fwd_regs(channels, TSEL_BIT)

### Startup Board

Set the Board IP address to your boards address.
Set the Receiver IP address to your computers interface address.

In [ ]:
BOARD = Board("hdsocv2_evalr2")
BOARD.start_server(str(READOUT_DIR))
BOARD_ADDRESS = ("192.168.22.58", 4660)
RECEIVER_ADDRESS = ("192.168.22.56", 4660)
SERVER_ADDRESS = BOARD.context.address
BOARD.connect_udp(BOARD_ADDRESS, RECEIVER_ADDRESS, SERVER_ADDRESS)
BOARD.initialize()

### Set DAC / Readout Channels

Set Default values for the readout channels and the DACs.

In [ ]:
BOARD.ext_bias.set_dacs(1804)
BOARD.readout.set_readout_channels([*range(BOARD.channels)])

In [ ]:
AnalogRegisters(BOARD).write("sst_to_digital_left", 20)
AnalogRegisters(BOARD).write("sst_to_digital_right", 20)

### Capture data - DAC-Trigger sweep

In [ ]:
from typing import Iterable
from tqdm import tqdm


def dac_trigger_scan(
    board: Board,
    dac_values: Iterable,
    reference_values: tuple[int, int],
    trigger_value: int,
    tsel=True,
):
    """Sweep the trigger values by holding the trigger valueand tref values static then sweep the DAC values"""
    set_references(board, reference_values)
    board.trigger.value = trigger_value

    set_ch_en(board, {ch: 1 for ch in range(board.channels)})
    set_tsel(board, {ch: tsel for ch in range(board.channels)})

    output = {}
    for dac_val in tqdm(dac_values):
        board.ext_bias.set_dacs(int(dac_val))  # Setting all DACs at the same time
        time.sleep(0.005)
        scalers = board.control.read_scalers([*range(board.channels)])
        output[int(dac_val)] = scalers

    now = datetime.datetime.now()
    filename_str = now.strftime("%Y%m%d_%H%M%S")
    filename = f"ref{reference_values[0]}-{reference_values[1]}_range{dac_values.start}-{dac_values.stop}-{dac_values.step}_{filename_str}.pkl"
    with open(DATA_DIR / filename, "wb") as file:
        pickle.dump(output, file)
    print(f"Saved data to: {DATA_DIR / filename}")
    return output

### Run scan

For a complete scan, run for all reference values 0 to 16.

In [ ]:
# Step size of 5 helps to find the trigger threshold, it's possible to miss if the steps are too big.
rng = range(1000, 3000, 5)

# Will set the trigger value close to 1150mV, which is close to the default ext DAC position.
ref_val = (7, 7)

# Set trigger value slightly above middle to avoid an issue with powers of 2.
trig_val = 130

DATA = dac_trigger_scan(BOARD, rng, ref_val, trig_val)

### Load data

Don't load `DATA` if you ran the code above.

In [ ]:
# If you ran the above capture, DON'T load the data.
filename = "filename.pkl"  # Replace with the real name

with open(DATA_DIR / filename, "rb") as file:
    DATA = pickle.load(file)

### Plot number of triggers vs external bias voltage

In [ ]:
D = {}
for k, v in DATA.items():
    D[str(k)] = v.values()

df = pl.dataframe.DataFrame(D)

In [ ]:
plt.figure(figsize=(12, 8))
for ch in range(64):
    plt.plot([x * 2500 / 4095 for x in DATA.keys()], df.row(ch))
# plt.xlim(1850, 1950)
plt.title(f"Fixed trigger vs ext bias, all channels, ref: {ref_val}, trig: {trig_val}")
plt.xlabel("Ext bias [mV]")
plt.ylabel("occurences")
plt.grid()